In [1]:
import os
import datetime
import json
import twython
import json

In [2]:
import pandas as pd
import numpy as np 

In [3]:
from twython import Twython
from contextlib import suppress
from requests_oauthlib import OAuth1Session
from apscheduler.schedulers.background import BackgroundScheduler as Scheduler

In [24]:
# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = '3mnbKAJQ1mTqqCUkflnMPU7Yd'
credentials['CONSUMER_SECRET'] = 'u1S9w46A0bswd3famLV0AE20MJng022uEu2P6s4RpQ5NwVlblY'
credentials['ACCESS_KEY'] = '1308202324725121025-Bea5jYYdFz150u4Unj9n1yvoK4f66p' 
credentials['ACCESS_SECRET'] = 'DzCPzqqyOlcjKxgduKcw6OEIqsucz1jIrl1Zh8PxvUsEm'

# Save the credentials object to file
with open("data/twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [27]:
# Load credentials from json file
with open("data/twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [28]:
# getting the header for our dataframe so we have something to append to 
startdf = pd.read_csv('df')
# when our dataframe is created the index is unnamed, when you export it is renamed so we need to fix it so our df realizes it's the same 
startdf = startdf.head(0).rename(columns={'Unnamed: 0':''})
#assign our id as the most recent tweet id to start so that we get everything after it 
id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']

In [29]:
def myfn(q, name):
    # import global variables we will be using this function to alter
    global id_
    global startdf 
    # Create our query
    query = {'q': q,
            'result_type': 'recent',
            'count': 100,
            'lang': 'en',
            'max_id': id_,
            'tweet_mode' : 'extend',
            'entities': {
                "hashtags": [],
                "symbols": [],
                "user_mentions": []
            }
            }
    #build our query into a dictionary to easily turn into dataframe 
    dict_ = {'user': [], 'user_id':[], 'post_id': [], 'text': [], 'favorite_count': [], 'hashtags':[], 'symbols':[], 'user_mentions':[], 
             'retweet_count':[]}
    for status in python_tweets.search(**query)['statuses']:
        dict_['user'].append(status['user']['screen_name'])
        dict_['user_id'].append(status['user']['id'])
        dict_['text'].append(status['text'])
        dict_['hashtags'].append(status['entities']['hashtags'])
        dict_['symbols'].append(status['entities']['symbols'])
        dict_['user_mentions'].append(status['entities']['user_mentions'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['retweet_count'].append(status['retweet_count'])
        dict_['post_id'].append(status['id'])
    # put data in a DataFrame to work with it easier
    df = pd.DataFrame(dict_)
    #removing our vairables stuck in dictionaries within our dataframe and give them their own columns
    df['mentions'] = ' '
    df['hashtag'] = ' '
    for i in range(len(df)):
        for j in range(len(df.at[i, 'user_mentions'])):
            try:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions']) + ' ' + str(df.at[i, 'user_mentions'][j]['id'])
            except:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions'])
        for k in range(len(df.at[i, 'hashtags'])):
            try:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag']) + ' ' + str(df.at[i, 'hashtags'][k]['text'])
            except:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag'])
    # get rid of columns that we took everything we need from 
    df = df.drop(['user_mentions', 'hashtags'], 1)
    startdf = startdf.append(df)
    # not sure why id_ doesn't want to change unless reset but do it because it works and can't hurt
    id_ = None
    # get oldest id_ (lowest number) and subtract one so that that you don't reinclude it in the next search
    # reset id_ to be oldest
    try: 
        id_ = int(df['post_id'].sort_values(ascending=True).to_frame().reset_index()['post_id'][0]) - 1 
    except:
        id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']
    return startdf, id_

In [30]:
# 'astronomy', 'Starship', 'mars', 'curiosityrover', 'oppertunityrover', 'starlink', 'falconheavy', 'sls','ESA', 'NASA', 'spacex', 'virgingalactic', 'virginorbit', 'JAXA', 'Roscosmos', 'artemis', 
# 'starliner', 'blueorigin', 'spacetravel'

In [31]:
query_list = ['marswebcam', 'falcon9', 'nasa_app', 'universe', 'cosmos', 'iss', 'climate', 'internationalspacestation', 'futurism', 'starliner', 'blueorigin', 'spacetravel', 'astronomy', 'mars', 'curiosityrover', 'oppertunityrover', 'starlink', 'falconheavy', 'sls','ESA', 'NASA', 'spacex', 'virgingalactic', 'virginorbit', 'JAXA', 'Roscosmos', 'areospace']

In [32]:
# eventual goal is to use those loop to do all of our searches
end_item = datetime.datetime.now()
id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']

for item in query_list:
    #reset id_ to be most recent tweet for each item
    id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']
    # for each item, reset the start time to the end of the last item 
    start_time = end_item + datetime.timedelta(seconds=1)
    # set the end time to be 90 min after you start, gathering 50 tweets every 100 seconds for 2,700 tweets from each item in list
    end_item = end_item + datetime.timedelta(seconds=1500)
    #print start and end times so I know when the program will be finished and where it should be at 
    print(f'starting {item} time : {start_time.strftime("%H:%M:%S")}, ending time: {end_item.strftime("%H:%M:%S")}')
    #occassionally getting key/value errors of 0 that I cannot find the cause of, but do not affect anything in how our dataframe
    #is made, I just don't want them printing 
    with suppress(KeyError, ValueError):
        #initiate scheduler
        sch = Scheduler()
        #add our function and items, we're doing 50/10 seconds 
        sch.add_job(myfn, 'interval', (item, startdf), seconds= 5, start_date=start_time, end_date=end_item)
        #start scheduler
        sch.start()

starting marswebcam time : 22:21:02, ending time: 22:46:01
starting falcon9 time : 22:46:02, ending time: 23:11:01
starting nasa_app time : 23:11:02, ending time: 23:36:01
starting universe time : 23:36:02, ending time: 00:01:01
starting cosmos time : 00:01:02, ending time: 00:26:01
starting iss time : 00:26:02, ending time: 00:51:01
starting climate time : 00:51:02, ending time: 01:16:01
starting internationalspacestation time : 01:16:02, ending time: 01:41:01
starting futurism time : 01:41:02, ending time: 02:06:01
starting starliner time : 02:06:02, ending time: 02:31:01
starting blueorigin time : 02:31:02, ending time: 02:56:01
starting spacetravel time : 02:56:02, ending time: 03:21:01
starting astronomy time : 03:21:02, ending time: 03:46:01
starting mars time : 03:46:02, ending time: 04:11:01
starting curiosityrover time : 04:11:02, ending time: 04:36:01
starting oppertunityrover time : 04:36:02, ending time: 05:01:01
starting starlink time : 05:01:02, ending time: 05:26:01
star

In [42]:
len(startdf)

649309

In [40]:
startdf

,,user,user_id,text,favorite_count,retweet_count,mentions,hashtag,post_id,symbols
0,NaN,rnitsch,55369958,RT @esamarswebcam: New VMC Images direct from ...,0,3,158283753,marswebcam,1379577401626263560,[]
1,NaN,AK9dj,1303105529892364289,RT @esamarswebcam: New VMC Images direct from ...,0,1,158283753,marswebcam,1379575449290895360,[]
2,NaN,esamarswebcam,158283753,New VMC Images direct from Mars! 22 images tak...,4,1,,marswebcam,1379572699794640902,[]
3,NaN,nasafan_,1274008106083770368,RT @esamarswebcam: New VMC Images direct from ...,0,1,158283753,marswebcam,1379572287725264898,[]
4,NaN,esamarswebcam,158283753,New VMC Images direct from Mars! 74 images tak...,3,1,,marswebcam,1379572106430709760,[]
...,...,...,...,...,...,...,...,...,...,...
95,NaN,RichTPar,199730805,"RT @ISAS_JAXA_EN: From this week, the Hayabusa...",0,5,1354997580346716164,,1377462589634977793,[]
96,NaN,Grace68738590,1219765988452782080,@Thom_astro @SpaceX @NASA_Astronauts @esa @JAX...,0,0,437520768 34743251 43166813 21436960 104989762,,1377462519929872394,[]
97,NaN,XKblo,67767462,RT @jaxa_wdc: Navigating satellites away from ...,0,27,4529166736,,1377462058334228483,[]
98,NaN,Tokolose,950701406,@ArduinoMakes @antonioastro_ @TJ_Cooney Sure a...,0,0,945002912023023616 1214972829449539584 11339...,,1377460988006531074,[]


In [43]:
newstartdf = startdf[['user', 'user_id', 'text', 'favorite_count', 'retweet_count',
                   'mentions', 'hashtag', 'post_id']]

In [44]:
# df = df.append(newstartdf)
# df = df.drop_duplicates(subset='post_id')
# df = df.sort_values(by='favorite_count', ascending=False).drop_duplicates(subset='text', keep='first')
# df = df.reset_index()
# print(len(df))
# #171476

NameError: name 'df' is not defined

In [ ]:
df['hashtag'].value_counts().to_frame().head(50)

In [45]:
startdf.to_csv('beeep')

In [ ]:
# since every time we loop through it starts a new index, there are now multiple entries for each index, we need to fix this
# and the easiest way is to just reset the index once it finishes running 
startdf = startdf.drop_duplicates(subset='post_id')
startdf = startdf.reset_index()

In [ ]:
# don't need these columns anymore
startdf = startdf.drop(['index', '', 'symbols'], 1)

In [ ]:
# in order to add our strings for our hashtags and mentions together we had to make the entire column a string, sinlucding empty cells
# here we go through and replace empty cells with nan values so pandas will read them as being empty instead of a string
for i in range(len(startdf)):
    if startdf.at[i, 'mentions'] == ' ':
        startdf.at[i, 'mentions'] = np.nan
    else: 
        pass 
    if startdf.at[i, 'hashtag'] == ' ':
        startdf.at[i, 'hashtag'] = np.nan
    else: 
        pass 